In [ ]:
!pip install tensorflow_io
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas
import tensorflow_datasets as tfds
import time
import librosa.display as lidp
import tensorflow_io as tfio
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Conv1DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.constraints import Constraint
from numpy import expand_dims
from numpy import mean
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from scipy.linalg import sqrtm
from sklearn.neighbors import NearestNeighbors
from IPython import display

In [ ]:
import json

In [ ]:
data, info = tfds.load('nsynth', try_gcs=True, split='train', with_info=True)
assert isinstance(data, tf.data.Dataset)
#get data

In [ ]:
data = data.shuffle(100*16)

In [ ]:
testdata, info = tfds.load('nsynth', try_gcs=True, split='test', with_info=True)
assert isinstance(testdata, tf.data.Dataset)
print(len(testdata))
test_x = np.array([])
test_y = np.array([])
count = 0
for i in data.take(1000):
	# select images
  sound = i["audio"]
  sound = sound[0:16384].numpy()
  test_x = np.append(test_x,sound) 
  # generate class labels, -1 for 'real'
  label = i["instrument"]["family"]
  c = label.numpy()
  #print(c)
  reallabel = np.zeros([11])
  reallabel[c] = 1
  test_y = np.append(test_y,reallabel)
  print(count)
  count += 1
print(test_x)
print(test_y)

In [ ]:
test_x = np.reshape(test_x,(1000,16384,1))
test_y = np.reshape(test_y,(1000,11))
print(test_x.shape)
print(test_y.shape)

In [ ]:
is_avg, is_std = calculate_inception_score(test_x)
print('score', is_avg, is_std)

In [ ]:
r_x = np.array([])
r_y = np.array([])
count = 0
for i in testdata.take(1000):
	# select images
  sound = i["audio"]
  sound = sound[0:16384].numpy()
  r_x = np.append(r_x,sound) 
  print(count)
  count += 1

In [ ]:
r_x = np.reshape(r_x,(1000,16384,1))

In [ ]:
is_avg, is_std = calculate_inception_score(r_x)
print('score', is_avg, is_std)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#login your dirve so that you can save model data for later training

In [ ]:
def generate_real_samples(dataset, n_samples):
	# choose random instances
	batch = next(dataset)
	# select images
	sound = batch["audio"]
	sound = sound[:,0:16384]
	X = tf.reshape(sound,[sound.shape[0],16384,1])
	# generate class labels, -1 for 'real'
	label = batch["instrument"]["family"]
	c = label.numpy()
	reallabel = np.zeros([label.shape[0],11])
	reallabel[np.arange(reallabel.shape[0]),c] = 1
	#print(reallabel)
	y = tf.convert_to_tensor(reallabel)
	return X, y

In [ ]:
def calculate_inception_score(images, n_split=10, eps=1E-16):
	# load model
	model = keras.models.load_model('/content/gdrive/My Drive/rmodel')
	# predict class probabilities for images
	yhat = model.predict(images)
	# enumerate splits of images/predictions
	#print(yhat)
	scores = list()
	n_part = floor(images.shape[0] / n_split)
	for i in range(n_split):
		# retrieve p(y|x)
		ix_start, ix_end = i * n_part, i * n_part + n_part
		p_yx = yhat[ix_start:ix_end]
		# calculate p(y)
		p_y = expand_dims(p_yx.mean(axis=0), 0)
		# calculate KL divergence using log probabilities
		kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
		# sum over classes
		sum_kl_d = kl_d.sum(axis=1)
		# average over images
		avg_kl_d = mean(sum_kl_d)
		# undo the log
		is_score = exp(avg_kl_d)
		# store
		scores.append(is_score)
	# average across images
	is_avg, is_std = mean(scores), std(scores)
	return is_avg, is_std

In [ ]:
images = tf.random.normal(shape=(5000, 100))
print('loaded', images.shape)
generator = keras.models.load_model('/content/gdrive/My Drive/amodel')
audio = generator.predict(images)
#print(audio)
# calculate inception score
is_avg, is_std = calculate_inception_score(audio)
print('score', is_avg, is_std)

In [ ]:
images = tf.random.normal(shape=(5000, 100))
print('loaded', images.shape)
generator = keras.models.load_model('/content/gdrive/My Drive/bmodel')
audio = generator.predict(images)
print(audio)
# calculate inception score
is_avg, is_std = calculate_inception_score(audio)
print('score', is_avg, is_std)

In [ ]:
images = tf.random.normal(shape=(5000, 100))
print('loaded', images.shape)
generator = keras.models.load_model('/content/gdrive/My Drive/cmodel')
audio = generator.predict(images)
print(audio)
# calculate inception score
is_avg, is_std = calculate_inception_score(audio)
print('score', is_avg, is_std)

In [ ]:
def invert_spectra_griffin_lim(X_mag, nfft, nhop, ngl):
    x = tf.signal.inverse_stft(X_mag,nfft,nhop)
    x = x[:, :16384]
    #print(x.shape)
    return x

In [ ]:
def f_to_t(X_norm ,ngl=16):
  x = invert_spectra_griffin_lim(X_norm,256,128,ngl)
  x = tf.reshape(x, [-1, 16384, 1])
  #print(x.shape)
  return x

In [ ]:
images = tf.random.normal(shape=(5000, 100))
print('loaded', images.shape)
generator = keras.models.load_model('/content/gdrive/My Drive/dmodel')
predictions = generator.predict(images)
a = predictions[:,:,:,0]
b = predictions[:,:,:,1]
r = tf.complex(a, b)
audio = f_to_t(r)
#print(audio)
# calculate inception score
is_avg, is_std = calculate_inception_score(audio)
print('score', is_avg, is_std)

In [ ]:
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

In [ ]:
model = keras.models.load_model('/content/gdrive/My Drive/rmodel')
print(model.summary())
model2 = keras.models.Model(inputs=model.input, outputs=model.get_layer("flatten").output)
print(model2.summary())

In [ ]:
images1 = test_x
print(images1.shape)
generator = keras.models.load_model('/content/gdrive/My Drive/amodel')
z = tf.random.normal(shape=(2, 100))
images2 = generator.predict(z)
# fid between images1 and images1
fid = calculate_fid(model2, images1, images1)
print('FID (same): %.3f' % fid)
# fid between images1 and images2
fid = calculate_fid(model2, images1, images2)
print('FID (different): %.3f' % fid)

In [ ]:
def nn_dist(train_set, query_set, exclude_self):
  # Flatten
  train_set = np.reshape(train_set, [train_set.shape[0], -1])
  query_set = np.reshape(query_set, [query_set.shape[0], -1])

  # Create and query model
  print('Creating model')
  start = time.time()
  model = NearestNeighbors(n_neighbors=2 if exclude_self else 1, algorithm='ball_tree').fit(train_set)
  print('Took {} seconds'.format(time.time() - start))

  print('Querying model')
  start = time.time()
  dists, _ = model.kneighbors(query_set)
  print('Took {} seconds'.format(time.time() - start))

  # If specified, exclude first nearest neighbor (duplicate) if it is nonzero
  if exclude_self:
    dists_excluded = []
    for dist0, dist1 in dists:
      if dist0 == 0:
        dists_excluded.append(dist1)
      else:
        dists_excluded.append(dist0)
    dists = dists_excluded
  else:
    dists = dists[:, 0]

  return np.mean(dists), np.std(dists)

In [ ]:
generator = keras.models.load_model('/content/gdrive/My Drive/amodel')
z = tf.random.normal(shape=(1000, 100))
query_set = generator.predict(z)
train_set = test_x

In [ ]:
generator = keras.models.load_model('/content/gdrive/My Drive/bmodel')
z = tf.random.normal(shape=(1000, 100))
query_set = generator.predict(z)
train_set = test_x

In [ ]:
generator = keras.models.load_model('/content/gdrive/My Drive/cmodel')
z = tf.random.normal(shape=(1000, 100))
query_set = generator.predict(z)
train_set = test_x

In [ ]:
generator = keras.models.load_model('/content/gdrive/My Drive/dmodel')
z = tf.random.normal(shape=(1000, 100))
predictions = generator.predict(z)
a = predictions[:,:,:,0]
b = predictions[:,:,:,1]
r = tf.complex(a, b)
query_set = f_to_t(r)
train_set = test_x

In [ ]:
query_set = r_x
train_set = test_x

In [ ]:
mean, std = nn_dist(query_set, query_set, True)
print('Similarity: {} +- {}'.format(mean, std))

In [ ]:
mean, std = nn_dist(train_set, query_set, False)
print('Similarity: {} +- {}'.format(mean, std))